In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore") 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [2]:
df_customer = pd.read_csv("Customers.csv")
df_customer.head()

,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15


In [3]:
df_products = pd.read_csv("Products.csv")
df_products.tail()

,ProductID,ProductName,Category,Price
95,P096,SoundWave Headphones,Electronics,307.47
96,P097,BookWorld Cookbook,Books,319.34
97,P098,SoundWave Laptop,Electronics,299.93
98,P099,SoundWave Mystery Book,Books,354.29
99,P100,HomeSense Sweater,Clothing,126.34


In [4]:
df_transactions = pd.read_csv("Transactions.csv")
df_transactions.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68


Converting `SignupDate` and `TransactionDate` to datetime format

In [5]:
df_customer['SignupDate'] = pd.to_datetime(df_customer['SignupDate'])
df_transactions['TransactionDate'] = pd.to_datetime(df_transactions['TransactionDate'])

In [6]:
print("Customer dataset datatypes")
print(df_customer.dtypes)
print(40 * "-")
print("Products dataset datatypes")
print(df_products.dtypes)
print(40 * "-")
print("Transaction dataset datatypes")
print(df_transactions.dtypes)

Customer dataset datatypes
CustomerID              object
CustomerName            object
Region                  object
SignupDate      datetime64[ns]
dtype: object
----------------------------------------
Products dataset datatypes
ProductID       object
ProductName     object
Category        object
Price          float64
dtype: object
----------------------------------------
Transaction dataset datatypes
TransactionID              object
CustomerID                 object
ProductID                  object
TransactionDate    datetime64[ns]
Quantity                    int64
TotalValue                float64
Price                     float64
dtype: object


In [7]:
df_transactions.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68


Merging `transaction` and `products` datasets using a left join on `product_id`, and the new dataset is `df_merged`.

In [8]:
df_merged = df_transactions.merge(df_products, on="ProductID", how="left")
df_merged.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68


In [9]:
if 'Price_x' in df_merged.columns and 'Price_y' in df_merged.columns:
    if df_merged['Price_x'].equals(df_merged['Price_y']):
        df_merged.drop(columns=['Price_y'], inplace=True)
        df_merged.rename(columns={'Price_x': 'Price'}, inplace=True)

In [10]:
df_merged.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,ProductName,Category
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,ComfortLiving Bluetooth Speaker,Electronics
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,ComfortLiving Bluetooth Speaker,Electronics


In [11]:
print(df_merged.shape)
print(df_merged.size)

(1000, 9)
9000


Performing `groupby` and aggregation on columns such as:

- Total count of `transaction_id`
- Total Quantity Purchased
- Last transaction date
- Most bought product
- Most frequent product category


In [12]:
customer_features = df_merged.groupby('CustomerID').agg(
    total_spend=('TotalValue', 'sum'),
    avg_transaction_value=('TotalValue', 'mean'),
    transaction_count=('TransactionID', 'count'),
    total_quantity = ('Quantity','sum'),
    last_transaction=('TransactionDate', 'max'),
    preferred_product = ('ProductName',lambda x:x.mode()[0]),
    preferred_category=('Category', lambda x: x.mode()[0])
).reset_index()

In [13]:
customer_features.head()

,CustomerID,total_spend,avg_transaction_value,transaction_count,total_quantity,last_transaction,preferred_product,preferred_category
0,C0001,3354.52,670.904,5,12,2024-11-02 17:04:16,ActiveWear Smartwatch,Electronics
1,C0002,1862.74,465.685,4,10,2024-12-03 01:41:41,BookWorld Cookware Set,Clothing
2,C0003,2725.38,681.345,4,14,2024-08-24 18:54:04,ActiveWear Cookware Set,Home Decor
3,C0004,5354.88,669.360,8,23,2024-12-23 14:13:52,ActiveWear Cookware Set,Books
4,C0005,2034.24,678.080,3,7,2024-11-04 00:30:22,ActiveWear Cookware Set,Electronics


In [14]:
customer_features.shape

(199, 8)

Merging `customer` dataset with the new data`customer_features` on `customer_id` using a left join.

In [15]:
df_customer = df_customer.merge(customer_features, on='CustomerID', how='left')
df_customer.head()

,CustomerID,CustomerName,Region,SignupDate,total_spend,avg_transaction_value,transaction_count,total_quantity,last_transaction,preferred_product,preferred_category
0,C0001,Lawrence Carroll,South America,2022-07-10,3354.52,670.904,5.0,12.0,2024-11-02 17:04:16,ActiveWear Smartwatch,Electronics
1,C0002,Elizabeth Lutz,Asia,2022-02-13,1862.74,465.685,4.0,10.0,2024-12-03 01:41:41,BookWorld Cookware Set,Clothing
2,C0003,Michael Rivera,South America,2024-03-07,2725.38,681.345,4.0,14.0,2024-08-24 18:54:04,ActiveWear Cookware Set,Home Decor
3,C0004,Kathleen Rodriguez,South America,2022-10-09,5354.88,669.360,8.0,23.0,2024-12-23 14:13:52,ActiveWear Cookware Set,Books
4,C0005,Laura Weber,Asia,2022-08-15,2034.24,678.080,3.0,7.0,2024-11-04 00:30:22,ActiveWear Cookware Set,Electronics


Performed label encoding on `region`, `preferred_product`, and `preferred_category`.

In [16]:
encoder = LabelEncoder()
df_customer['Region'] = encoder.fit_transform(df_customer['Region'])
df_customer['preferred_product'] = encoder.fit_transform(df_customer['preferred_product'])
df_customer['preferred_category'] = encoder.fit_transform(df_customer['preferred_category'])

In [17]:
df_customer.head()

,CustomerID,CustomerName,Region,SignupDate,total_spend,avg_transaction_value,transaction_count,total_quantity,last_transaction,preferred_product,preferred_category
0,C0001,Lawrence Carroll,3,2022-07-10,3354.52,670.904,5.0,12.0,2024-11-02 17:04:16,9,2
1,C0002,Elizabeth Lutz,0,2022-02-13,1862.74,465.685,4.0,10.0,2024-12-03 01:41:41,16,1
2,C0003,Michael Rivera,3,2024-03-07,2725.38,681.345,4.0,14.0,2024-08-24 18:54:04,2,3
3,C0004,Kathleen Rodriguez,3,2022-10-09,5354.88,669.360,8.0,23.0,2024-12-23 14:13:52,2,0
4,C0005,Laura Weber,0,2022-08-15,2034.24,678.080,3.0,7.0,2024-11-04 00:30:22,2,2


Checking for columns with `NaN` values after performing the join.


In [18]:
# Check for columns with NaN values
nan_columns = df_customer.isna().sum()

# Filter columns that have NaN values
nan_columns = nan_columns[nan_columns > 0]

print("Columns with NaN values:")
print(nan_columns)


Columns with NaN values:
total_spend              1
avg_transaction_value    1
transaction_count        1
total_quantity           1
last_transaction         1
dtype: int64


In [19]:
df_customer[df_customer["CustomerID"] == "C0180"]

,CustomerID,CustomerName,Region,SignupDate,total_spend,avg_transaction_value,transaction_count,total_quantity,last_transaction,preferred_product,preferred_category
179,C0180,Amy Carpenter,0,2023-10-25,NaN,NaN,NaN,NaN,NaT,48,4


In [20]:
df_customer.shape

(200, 11)

In [21]:
df_customer = df_customer[df_customer["CustomerID"] != "C0180"]

df_customer.shape


(199, 11)

Extracted the number of days from the reference date of `SignupDate` and `LastTransactionDate`.

In [22]:
reference_date = pd.to_datetime('2024-12-31')

In [23]:
df_customer['days_from_last_transaction'] = (reference_date - df_customer['last_transaction']).dt.days

In [24]:
df_customer['days_from_signup_date'] = (reference_date - df_customer['SignupDate']).dt.days

print(df_customer[['CustomerID', 'days_from_signup_date']])

    CustomerID  days_from_signup_date
0        C0001                    905
1        C0002                   1052
2        C0003                    299
3        C0004                    814
4        C0005                    869
..         ...                    ...
195      C0196                    938
196      C0197                    651
197      C0198                   1038
198      C0199                    759
199      C0200                    569

[199 rows x 2 columns]


In [25]:
df_customer.head()

,CustomerID,CustomerName,Region,SignupDate,total_spend,avg_transaction_value,transaction_count,total_quantity,last_transaction,preferred_product,preferred_category,days_from_last_transaction,days_from_signup_date
0,C0001,Lawrence Carroll,3,2022-07-10,3354.52,670.904,5.0,12.0,2024-11-02 17:04:16,9,2,58,905
1,C0002,Elizabeth Lutz,0,2022-02-13,1862.74,465.685,4.0,10.0,2024-12-03 01:41:41,16,1,27,1052
2,C0003,Michael Rivera,3,2024-03-07,2725.38,681.345,4.0,14.0,2024-08-24 18:54:04,2,3,128,299
3,C0004,Kathleen Rodriguez,3,2022-10-09,5354.88,669.360,8.0,23.0,2024-12-23 14:13:52,2,0,7,814
4,C0005,Laura Weber,0,2022-08-15,2034.24,678.080,3.0,7.0,2024-11-04 00:30:22,2,2,56,869


In [26]:
df_customer = df_customer.drop(columns=['last_transaction'])

In [27]:
df_customer = df_customer.drop(columns=['SignupDate'])

In [28]:
df_customer.head()

,CustomerID,CustomerName,Region,total_spend,avg_transaction_value,transaction_count,total_quantity,preferred_product,preferred_category,days_from_last_transaction,days_from_signup_date
0,C0001,Lawrence Carroll,3,3354.52,670.904,5.0,12.0,9,2,58,905
1,C0002,Elizabeth Lutz,0,1862.74,465.685,4.0,10.0,16,1,27,1052
2,C0003,Michael Rivera,3,2725.38,681.345,4.0,14.0,2,3,128,299
3,C0004,Kathleen Rodriguez,3,5354.88,669.360,8.0,23.0,2,0,7,814
4,C0005,Laura Weber,0,2034.24,678.080,3.0,7.0,2,2,56,869


Performed Min-Max scaling only on the required columns.

In [29]:
columns_to_scale = ['Region', 'total_spend', 'avg_transaction_value', 'transaction_count','total_quantity','preferred_product', 'preferred_category', 'days_from_last_transaction','days_from_signup_date']

scaler = MinMaxScaler()

df_customer[columns_to_scale] = scaler.fit_transform(df_customer[columns_to_scale])

print(df_customer)

    CustomerID        CustomerName    Region  total_spend  \
0        C0001    Lawrence Carroll  1.000000     0.308942   
1        C0002      Elizabeth Lutz  0.000000     0.168095   
2        C0003      Michael Rivera  1.000000     0.249541   
3        C0004  Kathleen Rodriguez  1.000000     0.497806   
4        C0005         Laura Weber  0.000000     0.184287   
..         ...                 ...       ...          ...   
195      C0196         Laura Watts  0.333333     0.462684   
196      C0197    Christina Harvey  0.333333     0.174318   
197      C0198         Rebecca Ray  0.333333     0.080203   
198      C0199      Andrea Jenkins  0.333333     0.179098   
199      C0200         Kelly Cross  0.000000     0.441508   

     avg_transaction_value  transaction_count  total_quantity  \
0                 0.474336                0.4        0.354839   
1                 0.308940                0.3        0.290323   
2                 0.482751                0.3        0.419355   
3      

In [30]:
df_customer.head()

,CustomerID,CustomerName,Region,total_spend,avg_transaction_value,transaction_count,total_quantity,preferred_product,preferred_category,days_from_last_transaction,days_from_signup_date
0,C0001,Lawrence Carroll,1.0,0.308942,0.474336,0.4,0.354839,0.191489,0.666667,0.155125,0.842204
1,C0002,Elizabeth Lutz,0.0,0.168095,0.308940,0.3,0.290323,0.340426,0.333333,0.069252,0.979458
2,C0003,Michael Rivera,1.0,0.249541,0.482751,0.3,0.419355,0.042553,1.000000,0.349030,0.276377
3,C0004,Kathleen Rodriguez,1.0,0.497806,0.473092,0.7,0.709677,0.042553,0.000000,0.013850,0.757236
4,C0005,Laura Weber,0.0,0.184287,0.480120,0.2,0.193548,0.042553,0.666667,0.149584,0.808590


The code extracts specific customer features from the `df_customer` DataFrame and then calculates the cosine similarity between all customers based on these features, storing the result as a similarity matrix in `similarity_matrix`.


In [31]:
features = df_customer[['Region', 'total_spend', 'avg_transaction_value', 'transaction_count','total_quantity','preferred_product', 'preferred_category','days_from_last_transaction','days_from_signup_date']]
similarity_matrix = cosine_similarity(features)

The code converts the `CustomerID` column from the `df_customer` DataFrame into a list and stores it in the `customer_ids` variable. It then initializes an empty dictionary called `lookalike_map`


In [32]:
customer_ids = df_customer['CustomerID'].tolist()
lookalike_map = {}

The code iterates over the first 20 customer IDs (`customer_ids[:20]`) using `enumerate()` to get both the index `i` and the `cust_id`. For each customer:
- It retrieves the indices of the top 3 most similar customers (excluding the customer itself) from the `similarity_matrix`, using `argsort()` to sort the similarity scores in descending order (`[::-1]`), and then selects the top 3 indices (`[1:4]`).
- It creates a list of tuples, `similar_customers`, containing the customer ID and similarity score for each of the top 3 similar customers.
- It stores this list in the `lookalike_map` dictionary with the `cust_id` as the key.

This results in `lookalike_map` 


In [33]:
for i, cust_id in enumerate(customer_ids[:20]):  
    similar_indices = similarity_matrix[i].argsort()[::-1][1:4]  
    similar_customers = [(customer_ids[idx], similarity_matrix[i][idx]) for idx in similar_indices]
    lookalike_map[cust_id] = similar_customers

The code creates a DataFrame `lookalike_df` from the `lookalike_map` dictionary. It uses a list comprehension to extract the customer ID (`k`) and the corresponding top 3 similar customers and their similarity scores (`v[0][0]`, `v[0][1]`and so on) and organizes them into tuples. These tuples are then used to construct the DataFrame, with the following columns:
- `CustomerID`: The ID of the customer.
- `Lookalike1`, `Lookalike2`, `Lookalike3`: The IDs of the top 3 similar customers.
- `Score1`, `Score2`, `Score3`: The similarity scores for those customers.

Finally, it prints the `lookalike_df` to display the lookalike recommendations in tabular format.


In [34]:
lookalike_df = pd.DataFrame([(k, v[0][0], v[0][1], v[1][0], v[1][1], v[2][0], v[2][1]) for k, v in lookalike_map.items()],
                            columns=['CustomerID', 'Lookalike1', 'Score1', 'Lookalike2', 'Score2', 'Lookalike3', 'Score3'])


print(lookalike_df)


   CustomerID Lookalike1    Score1 Lookalike2    Score2 Lookalike3    Score3
0       C0001      C0184  0.985435      C0152  0.982952      C0192  0.976778
1       C0002      C0106  0.969802      C0134  0.969110      C0088  0.949481
2       C0003      C0035  0.978701      C0076  0.977160      C0031  0.976879
3       C0004      C0165  0.993468      C0155  0.986464      C0169  0.986395
4       C0005      C0007  0.981190      C0159  0.977080      C0166  0.963200
5       C0006      C0036  0.990946      C0187  0.985911      C0085  0.972219
6       C0007      C0005  0.981190      C0140  0.961081      C0186  0.952946
7       C0008      C0059  0.957966      C0124  0.950968      C0127  0.946723
8       C0009      C0160  0.961898      C0112  0.959324      C0192  0.957776
9       C0010      C0103  0.972474      C0199  0.957354      C0029  0.955428
10      C0011      C0174  0.993792      C0169  0.980093      C0155  0.979354
11      C0012      C0195  0.987336      C0039  0.980744      C0055  0.977546

In [35]:
lookalike_df.to_csv("Lookalike.csv", index=False)

print("Lookalike recommendations saved in Lookalike.csv")

Lookalike recommendations saved in Lookalike.csv
